In [1]:
from requests_html import HTMLSession
session = HTMLSession()

In [ ]:
import re
import pandas as pd

max_pagina = 6
datos = []

for pagina in range(1, max_pagina):
    url = "https://www.redcomputer.es/tarjetas-graficas-pc-108?page=" + str(pagina)
    print(url)
    r = session.get(url)

    linkGrafica = r.html.find("a.product-thumbnail")
    for link in linkGrafica:
        href = list(link.absolute_links)[0]
        print(href)
        r2 = session.get(href)
        precio = r2.html.find("span.product-price", first=True)
        if  precio is None or precio.text.strip() == "":
            precioFinal = "descatalogado"
        else:
            precioF2  = precio.text.replace("€","")
            precioF3  = precioF2.replace(",",".")
            precioF4 = precioF3.replace(".","").strip()
            precioCentimos = float(precioF4.strip()) * 100
            precioCentimosF =  str(precioCentimos).replace(".","").strip()
            precioFinal =  precioCentimosF[:-1]
            print(precioFinal)
        articulo = r2.html.find("h1.page-title > span", first=True)
        
        producto_info = {
            "URL": href,
            "Precio": precioFinal,
            "Articulo": articulo.text.strip()
        }
        memoria = r2.html.find("div.product-description   li")
        for memor in memoria:
            if any(palabra in memor.text for palabra in ["Memoria:", "Memoria de vídeo", "Memoria.", "Memoria de Video"]):
                separador = "." if "Memoria." in memor.text else ":"
                texto = memor.text.split(separador)
                if "GDDR"  in texto[1]:
                    partes = texto[1].split()
                    vram = partes[0].replace("GB","")        
                    if len(partes) == 3:   
                        tipo_vram = partes[2]
                    else:
                        tipo_vram =  partes[1]
                    print("VRAM:", vram)
                    print("Tipo VRAM:", tipo_vram)
                    producto_info["VRAM"] = vram.strip()
                    producto_info["Tipo VRAM"] = tipo_vram
            if re.search(r'\d+W',memor.text,re.IGNORECASE):
                texto  = memor.text.split(":")
                consumo =  texto[1].replace("W","")
                print("Consumo:", consumo)
                producto_info["Energia"] = consumo.strip()
            if "HDMI" in memor.text:
                texto  = memor.text.split("x")
                if texto[0].strip().isnumeric():
                    print("HDMI", ":" , texto[0])
                    producto_info["HDMI"] = texto[0]
                else:
                    print("HDMI", ":" , 1)
                    producto_info["HDMI"] = 1
            if "displayport" in memor.text.lower():
                texto  = memor.text.split("x")
                if texto[0].strip().isnumeric():
                    print("DisplayPort", ":" , texto[0])
                    producto_info["DisplayPort"] = texto[0]
                else:
                    print("DisplayPort", ":" , 1)
                    producto_info["DisplayPort"] = 1
        datos.append(producto_info)
df =  pd.DataFrame(datos)
            

Pagina 1: https://www.redcomputer.es/tarjetas-graficas-pc-108?page=1
Enlace del producto: https://www.redcomputer.es/sparkle-intel-arc-a750-orc-8gb-gddr6-19327.html
2109000
Precio: <Element 'span' itemprop='price' class=('product-price',) content='210.9'>
Marca: Sparkle Intel Arc A750 ORC 8GB GDDR6
VRAM: 8
Tipo VRAM: GDDR6
Consumo:  225
HDMI : 1 
DisplayPort : 3 
Enlace del producto: https://www.redcomputer.es/gigabyte-radeon-rx-7600-xt-gaming-oc-16gb-gddr6-14118.html
3458900
Precio: <Element 'span' itemprop='price' class=('product-price',) content='345.89'>
Marca: Gigabyte Radeon RX 7600 XT Gaming OC 16GB GDDR6
VRAM: 16
Tipo VRAM: GDDR6
Consumo:  600
HDMI : 2 
DisplayPort : 2 
Enlace del producto: https://www.redcomputer.es/gigabyte-rtx-4060-ti-eagle-oc-8gb-dlss3-12855.html
4549000
Precio: <Element 'span' itemprop='price' class=('product-price',) content='454.9'>
Marca: Gigabyte RTX 4060 Ti Eagle OC 8GB DLSS3
VRAM: 8
Tipo VRAM: GDDR6
Consumo:  500
HDMI : 2 
DisplayPort : 2 
Enlace del

In [32]:
df.to_csv('tarjetas_graficas2.csv', index=False)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   URL          160 non-null    object
 1   Precio       160 non-null    object
 2   Articulo     160 non-null    object
 3   VRAM         151 non-null    object
 4   Tipo VRAM    151 non-null    object
 5   Energia      118 non-null    object
 6   HDMI         157 non-null    object
 7   DisplayPort  147 non-null    object
dtypes: object(8)
memory usage: 10.1+ KB
